In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
from pymc3 import Model, Normal, GaussianRandomWalk, HalfNormal
from pymc3 import NUTS, sample
from scipy import optimize
from pymc3 import traceplot
from pymc3 import summary
import theano.tensor as tt
import theano as T

df = pd.read_csv("../data/data-ss1.txt")
T = df.values[:,0].astype(np.float32)
Y = df.values[:,1].astype(np.float32)

n_times = len(df["X"].unique())

#shape = クラスの数の確率変数に、クラスの値を取るデータ数次元のベクトルを入れる操作がありますが
#その詳細な説明は(https://pymc-devs.github.io/pymc3/notebooks/GLM-hierarchical.html)参照

basic_model = Model()



#GaussianRandomWalkを使わない方法
with basic_model:
    #事前分布
    s_mu = HalfNormal('s_mu', sd=100) #隣接時刻の状態の誤差
    s_Y =  HalfNormal('s_Y', sd=100) #各時刻における状態と観測の誤差
    mu_0 = Normal('mu_0',mu=0, sd=100) #t=0初期状態
    mu_1 = Normal('mu_1',mu=0, sd=100) #t=1初期状態
    
    #誤差項
    e_mu = Normal('e_mu', mu=0, sd=s_mu, shape =n_times-2)
    
    mu = tt.zeros((n_times))
    mu = tt.set_subtensor(mu[0], mu_0)
    mu = tt.set_subtensor(mu[1], mu_1)
    for i in list(range(n_times-2)):
        mu = tt.set_subtensor(mu[i+2], 2*mu[i+1] - mu[i] + e_mu[i])
    #likelihood
    Y_obs = Normal('Y_obs', mu=mu, sd=s_Y, observed=Y)

    #サンプリング
    trace = sample(1000)
    summary(trace)

Auto-assigning NUTS sampler...
Initializing NUTS using advi...
Average ELBO = -93.235: 100%|██████████| 200000/200000 [01:27<00:00, 2285.87it/s]
Finished [100%]: Average ELBO = -93.04
100%|██████████| 1000/1000 [13:47<00:00,  3.34it/s]


mu_0:

  Mean             SD               MC Error         95% HPD interval
  -------------------------------------------------------------------
  
  11.065           0.803            0.074            [10.655, 11.599]

  Posterior quantiles:
  2.5            25             50             75             97.5
  |--------------|==============|==============|--------------|
  
  10.580         10.991         11.153         11.285         11.579


mu_1:

  Mean             SD               MC Error         95% HPD interval
  -------------------------------------------------------------------
  
  11.026           0.668            0.062            [10.729, 11.437]

  Posterior quantiles:
  2.5            25             50             75             97.5
  |--------------|==============|==============|--------------|
  
  10.691         10.989         11.096         11.209         11.430


e_mu:

  Mean             SD               MC Error         95% HPD interval
  ----------------------